# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [1]:
import numpy as np

In [2]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

$$\alpha^2$$

In [3]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(9.880544731176773, 10.0)

In [4]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4):
    pv_1 = 1*np.exp(-rf_rate*mat)    # compute the price of the default-free bond
    U = np.random.uniform(size=int(n_sample))
    default_time = -(1/def_rate)*np.log(U)
    value_1 = (default_time>mat)*pv_1
    value_2 = (default_time<=mat)*np.exp(-rf_rate*default_time)*recovery
    return value_1.mean()+value_2.mean()

# Call your function
print(corp_bond(mat, def_rate, rf_rate, recovery, n_sample))

# Find the mean and std by calling the function 100 times. 
a = np.array([corp_bond(mat, def_rate, rf_rate, recovery, n_sample) for i in range(100)])
print(a.std())
print(a.mean())

0.43737990276093974
0.002331644859489854
0.44071753955844445


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [5]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=int(n_sample/2))
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
print(default_time.mean())
# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

10.036150344930679


(10.000000000000002, 10.0)

In [9]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    pv_1 = 1*np.exp(-rf_rate*mat)    # compute the price of the default-free bond
    U = np.random.uniform(size=int(n_sample))
    default_time = -(1/def_rate)*np.log(U)
    if antithetic:
        U = np.random.uniform(size=int(n_sample/2))
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    if mean_match:
        default_time += 1/def_rate-default_time.mean()
    value_1 = (default_time>mat)*pv_1
    value_2 = (default_time<=mat)*np.exp(-rf_rate*default_time)*recovery
    return value_1.mean()+value_2.mean()

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both

a = np.array([corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=True, mean_match=False) for i in range(100)])
print('Only antithetic:', a.mean(), a.std())
a = np.array([corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=False, mean_match=True) for i in range(100)])
print('Only mean_match:', a.mean(), a.std())
a = np.array([corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample) for i in range(100)])
print('Both:', a.mean(), a.std())
a = np.array([corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=False, mean_match=False) for i in range(100)])
print('None:', a.mean(), a.std())

Only antithetic: 0.44051608843804246 0.0019186221085123753
Only mean_match: 0.4404904225319116 0.001459283957242274
Both: 0.4405995559662729 0.0014960273908339484
None: 0.4403961432365871 0.0023538944149068377


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [7]:
### Put the analytic expression for the corporate bond price
from scipy import integrate

In [19]:
func1 = lambda x: np.exp(-rf_rate*x) * recovery * np.exp(-def_rate*x)*def_rate if x<=10 else np.exp(-rf_rate*mat) * np.exp(-def_rate*x)*def_rate
pv = integrate.quad(func1, 0, 100000)
pv

(0.44040907131425105, 1.2028871488441635e-08)

The result is really close to my MC result above. And if we do not use the antithetic method and mean-matching method. the result is more close to the analytic value of the corporate bond